### Get reviews
- The root link i use here is the same as the book link, just need to add'/order/_votes/page/{page}' to it
- To the books which have more than one page, just change the page_number on the link
- put it inside a loop to get all reviews
- I create functions to do different things, it works faster 

In [1]:
from bs4 import BeautifulSoup ## load necessay libs
import numpy as np
import requests
import pandas as pd
import time


In [2]:
df = pd.read_csv(r'C:\GIT\waterstones_main\Data\all_extra_info.csv') # read csv to dataframe

In [3]:
review_df = df[df['review_links'].notnull()] ## filter the null values
review_df = review_df[['num_reviews', 'review_links']] #create new dataframe with new columns 'num_reviews,'review_links
review_df['pages'] = (review_df['num_reviews'].astype(float).sort_values() / 12).apply(np.ceil)  # count how many reviews each book has
review_df.sort_values(by='pages', ascending=False, inplace=True) # sort dataframe by number of pages, from most to least


- loop through each url to get all reviews on each page into a dict
- store it in a big list with each dictionary.
- load to dataframe
- save to pickle file


In [4]:
def find_text(bs_find): # find the text in HTML
    return bs_find.text if bs_find else None
    

def extract(url): ## from url to soup
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    return (url, soup)


def transform(soup): ###scrape the review details
    url, soup = soup[0], soup[1]
    reviews = soup.find_all('div',class_='review-list-item-main')

    for review in reviews:
        header = find_text(review.find('h2'))
        stars = review.find_all('div', class_='active')
        star_rating = len(stars)
        review_by = review.find('span', class_='review-meta').text[2:]
        review_date = review.find('div', class_='comment-bottom').div.text
        review_content = find_text(review.find('p', class_='p-medium')).strip()
        
        user_review = {'header':header,
                    'star_rating':star_rating,
                    'review_by':review_by,
                    'review_date':review_date,
                    'review_content':review_content,
                    'url': url}

        all_reviews.append(user_review)


                                      
def load(results): # get all_reviews make it as dataframe
    df = pd.DataFrame(results)
    return df


In [5]:
review_df

(8430, 3)

In [17]:
all_reviews = []

for row in review_df.itertuples():### in the for loop to loop through each book review link to get review details
    for page in range(1, int(row.pages)+1):
        soup = extract(row.review_links +  f'/order/_votes/page/{page}')
        try:
            transform(soup)

        except Exception as err:   ### write any errors into a file
            with open('errors.txt', 'a') as f:
                f.write(f'{err}\n')


load(all_reviews).to_pickle('all_reviews.pkl')

In [22]:
save_df = pd.read_pickle('all_reviews.pkl')

In [27]:
save_df.drop_duplicates(inplace=True)  ## drop the overlapped rows

In [29]:
save_df.to_pickle('books_reviews.pkl')### save to pickle file